In [ ]:
# 1. import everything you need (e.g., model, dataset, etc.)
# 2. Use Random Search to find the best hyperparams(e.g., batch_size, augmentation, optimizer, loss_fn, etc.)
# 3. Report the best train configuration
# 4. Save the best values (for hyperparams)
# 5. Use the found values in "train.ipynb" to train the model with them

# Code Cells for GoogleColab
> This way we can execute our code from github without any hassles, just:
>> 1. add all the packages needed (that is not in Colab) in "requirements.txt"
>> 2. The github repo must be public, if the Colab account (e.g., <golab_pro_owner>@gmail.com) has not been granted access to that repo.

In [ ]:
# # Mount the Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Clone the Github Repo
# !git clone https://github.com/tekboart/semantic-segmentaion-pytorch

Cloning into 'semantic-segmentaion-pytorch'...
remote: Enumerating objects: 7320, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 7320 (delta 106), reused 184 (delta 91), pack-reused 7121
Receiving objects: 100% (7320/7320), 255.31 MiB | 17.03 MiB/s, done.
Resolving deltas: 100% (231/231), done.


In [1]:
# %pwd

'/content'

In [2]:
# Go to the repo's main dir
# %cd semantic-segmentaion-pytorch/

/content/semantic-segmentaion-pytorch


In [3]:
# Install the required packages
# !pip install -r requirements.txt

In [5]:
# Allow Tensorboard to write to the tmp directory:
# !export TMPDIR=/tmp/$USER; mkdir -p $TMPDIR; tensorboard --logdir=~/ray_results

2023-07-14 10:11:50.897995: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-14 10:11:50.952853: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 10:11:51.735804: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/compat/__init__.py", line 42, in tf
    from tensorboard.compat import notf  # noqa: F401
ImportError: cannot import name 'notf' from 'tensorboard.compat' 

In [3]:
# reload modules
from importlib import reload

# load pretrained segmentation models (written in pytorch)
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from torch.utils.data import DataLoader

# torchvision
from torchviz import make_dot
import torchvision.transforms.v2 as TF

# torchmetrics
from torchmetrics.classification import Dice, BinaryJaccardIndex

# Ray Tune
import ray
from ray import tune
from ray.air import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.optuna import OptunaSearch
from ray.air.config import ScalingConfig

# Serialize/Deserialize Json files
import json

# Data Augmentation
import albumentations as A
import albumentations.augmentations.functional as F
from albumentations.pytorch import ToTensorV2

# get data/time with desired format
from datetime import datetime

time_format = "%Y.%m.%d@%H-%M-%S"

# work with images
import cv2
from PIL import Image

# slice Iterables and turn to GEN
from itertools import islice

# keep numpy use to a min
# as we store our torch.Tensors to GPU Vram but numpy in RAM (it only supports CPU)
import pandas as pd
import numpy as np
import random

# to have a progress bar
from tqdm import tqdm

# To use pretrained segmentation models (implement in PyTorch)

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
# uncomment if wan't to globally aloow sns to handle plot style
# it adds unwanted style (i.e., grid) to .imshow()
# so better to use it a context manager to style only what I want
# >>> with sns.axes_style('darkgrid'):
# >>>     plt.imshow(...)
# sns.set_theme(
#     context="notebook",
#     style="darkgrid",
#     palette="deep",
#     font="sans-serif",
#     font_scale=1,
#     color_codes=True,
#     rc={'axes.grid': False},
# )

# OS/File/Path management
import sys
import os

# Misc
from functools import partial

# load my custom Classes/Functions/etc.
from utils.training import fit_fn
from utils.dataset import get_loaders, SegmentaionDataset
from utils.models.unet import UnetScratch
from utils.visualization import (
    image_mask_plot,
    ImageAntiStandardize,
    plot_metrics
)
from utils.metrics import (
    AccuracyBinarySegment,
    DiceBinarySegment,
    JaccardBinarySegment,
    PrecisionBinarySegment,
    RecallBinarySegment,
    F1BinarySegment,
    DiceBCELoss,
)

/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

## Remove sources of non-determinism

In [ ]:
# use to seed the RNG for all devices (both CPU and CUDA).
torch.manual_seed(0)
# for custom operators, you might need to set python seed as well:
random.seed(0)
# If you or any of the libraries you are using rely on NumPy, you can seed the global NumPy RNG with:
np.random.seed(0)

# configure PyTorch to use deterministic algorithms instead of nondeterministic ones
# A CAVEAT: throws an error if an operation is known to be nondeterministic (and without a deterministic alternative).
# that is why we used warn_only to avoid raising Error
torch.use_deterministic_algorithms(True, warn_only=True)
# CUDA convolution determinism
torch.backends.cudnn.deterministic = True

# -- DataLoader
# make it deterministic but allow it to random data order
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

DATA_LOADER_GEN = torch.Generator()
DATA_LOADER_GEN.manual_seed(0)

# Define the Search Hyperparameters Space
> that is, the hyperparameters and their correspondign values to search for.

In [27]:
# Create the Config dict
config = {
    # 'lr': tune.loguniform(1e-4, 1e-3),  # step 1
    # 'lr': tune.loguniform(0.0005, 0.001),  # step 2
    'lr': tune.loguniform(0.0005, 0.0006),  # step 3
    # 'scheduler_factor': tune.choice([.5, .3, .1]),
    # 'batch_size': tune.choice([16, 32, 64]),
    'image_size': tune.choice([352]),  # sizes must be divisible by 32
    # 'pretrained_arch': tune.choice([smp.UnetPlusPlus, smp.FPN, smp.DeepLabV3Plus]),
    #TODO: Maybe remove ResNet152 as it's both (1) comp expensive and (2) can overfit very easily (considering its #params)
    # 'pretrained_encoder': tune.choice(['mobilenet_v2', 'timm-mobilenetv3_large_100', 'timm-efficientnet-b8']),
    # 'loss_fn': tune.choice([nn.BCEWithLogitsLoss(), DiceBCELoss(from_logits=True)]),
    # 'train_augmentation': tune.choice([
    #     #TODO: As my last model's val & test metrics were very different, then maybe I need stronger augmentations for the model to generalize better.
    #     # We didn't included the resize/Flip/etc. that we know would cause no problem. so:
    #     # Just use this A.Compose([*hyper_config['train_augmentation'], A.HorizontalFlip(...), A.Resize(..), etc.])
    #     [
    #         A.Rotate(limit=5, p=0.5),  # Use only when the img_height==img_width
    #         A.RandomRotate90(p=0.5),  # Use only when the img_height==img_width
    #         A.Transpose(p=0.5),  # Use only when the img_height==img_width
    #         # A.CenterCrop(
    #             #TODO: Remember to add vars image_height & image_width in the train func
    #             # int(0.9 * image_height),
    #             # int(0.9 * image_width),
    #             # p=0.3,
    #         # ),
    #         A.ColorJitter(
    #             brightness=0.3,
    #             contrast=0.05,
    #             saturation=0.1,
    #             hue=0.05,
    #             p=0.8,
    #             # always_apply=True,
    #         ),
    #         A.ImageCompression(quality_lower=90, quality_upper=100, p=0.3),
    #         A.RGBShift(
    #             r_shift_limit=5,
    #             g_shift_limit=5,
    #             b_shift_limit=2,
    #             p=0.8,
    #             # always_apply=True,
    #         ),
    #     ],
    #     [
    #         A.Rotate(limit=5, p=0.5),  # Use only when the img_height==img_width
    #         A.RandomRotate90(p=0.5),  # Use only when the img_height==img_width
    #         A.Transpose(p=0.5),  # Use only when the img_height==img_width
    #         A.ColorJitter(
    #             brightness=0.3,
    #             contrast=0.05,
    #             saturation=0.1,
    #             hue=0.05,
    #             p=0.8,
    #             # always_apply=True,
    #         ),
    #         A.RGBShift(
    #             r_shift_limit=5,
    #             g_shift_limit=5,
    #             b_shift_limit=2,
    #             p=0.8,
    #             # always_apply=True,
    #         ),
    #     ],
    #     [
    #         # means use no fancy Augmentation. Just some flip and 90deg rotations.
    #     ]
    # ])

}

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device used for calculation (CPU\Cuda):", device)

Device used for calculation (CPU\Cuda): cuda:0


# Create The Model Func
> Everything needed for trainin a model needs to be within its scope (should not use any var from outside)

In [28]:
# Create a model_creator_for_hyperserach to create + Compile a model (to be ready to be trained)
# TODO: Can I make it reusable for other models as well?
# A: I don't think
def model_creator(config):
    """
    instantiate + Compile + train (aka .fit()) a network

    Parameters
    ----------
    hyper_config: dict
        Includes all the hyperparmeters needed to be tuned.
    """
    DATA_LOADER_GEN = torch.Generator()
    DATA_LOADER_GEN.manual_seed(0)

    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device("cuda:0")
    # print("Device used for calculation (CPU\Cuda):", device)

    # Load all the needed variables (Ray Tune asks all the used variables to be inside this func scope)
    hyper_params = {
        # "device": str(device),
        "lr": 1e-3,
        "epochs": 10,  #XXX: Set the #epochs to at least 10 for the real run in Colab
        "batch_size": 16,
        "num_workers": 2,
        # Use Height == Width to use 90-rotations/transpose in data aug
        "image_height": 352,
        "image_width": 352,
        "input_channels": 3,
        "num_classes": 1,
        "data_format": "channels_first",
        "pin_mem": True,
        "scheduler_step": 5,
        "scheduler_factor": 0.5,
        "train_img_dir": os.path.join("data", "traincrop", "img"),
        "train_mask_dir": os.path.join("data", "traincrop", "mask"),
        "val_img_dir": os.path.join("data", "valcrop", "img"),
        "val_mask_dir": os.path.join("data", "valcrop", "mask"),
        "pretrained_model_encoder": "timm-mobilenetv3_large_100",
    }

    # Load a Pretrained Model
    # TODO: Load the preprocess_fn before, so no need to create it for each run
    # model_arch = config["pretrained_arch"]
    model_arch = smp.UnetPlusPlus

    # create the model
    model = model_arch(
        # choose encoder
        # encoder_name=config["pretrained_encoder"],
        encoder_name=hyper_params["pretrained_model_encoder"],
        # use `imagenet` pre-trained weights for encoder initialization
        encoder_weights="imagenet",
        # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        in_channels=hyper_params["input_channels"],
        # model output channels (number of classes in your dataset)
        classes=hyper_params["num_classes"],
    )

    preprocess_input = get_preprocessing_fn(
        # config["pretrained_encoder"], pretrained="imagenet"
        hyper_params["pretrained_model_encoder"], pretrained="imagenet"
    )

    # Create Data Augmentation
    train_transform = A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Rotate(limit=5, p=0.5),  # Use only when the img_height==img_width
            A.RandomRotate90(p=0.5),  # Use only when the img_height==img_width
            A.Transpose(p=0.5),  # Use only when the img_height==img_width
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.0, p=0.5),
            A.CenterCrop(
                int(0.95 * config["image_size"]),
                int(0.95 * config["image_size"]),
                p=0.3,
            ),
            A.ColorJitter(
                brightness=0.0,
                contrast=0.0,
                saturation=0.1,
                hue=0.05,
                p=0.8,
            ),
            A.ImageCompression(quality_lower=90, quality_upper=100, p=0.1),
            A.RGBShift(
                r_shift_limit=5,
                g_shift_limit=5,
                b_shift_limit=2,
                p=0.8,
            ),
            A.Resize(
                height=config["image_size"], width=config["image_size"]
            ),
        ],
    )

    # we don't want TTA, just some resize, normalization, etc.
    val_transform = A.Compose(
        [
            A.Resize(height=config["image_size"], width=config["image_size"]),
        ],
    )

    # Create the Datasets
    train_ds = SegmentaionDataset(
        image_dir=hyper_params["train_img_dir"],
        mask_dir=hyper_params["train_mask_dir"],
        transform=train_transform,
        preprocess_fn=preprocess_input,
        mask_suffix="",
        # subset=[0, 10],
    )
    val_ds = SegmentaionDataset(
        image_dir=hyper_params["val_img_dir"],
        mask_dir=hyper_params["val_mask_dir"],
        transform=val_transform,
        preprocess_fn=preprocess_input,
        mask_suffix="",
        # subset=[0, 5],
    )
    train_loader, val_loader = get_loaders(
        train_ds,
        val_ds,
        # batch_size=config["batch_size"],
        batch_size=hyper_params["batch_size"],
        num_workers=hyper_params["num_workers"],
        pin_memory=hyper_params["pin_mem"],
        worker_init_fn=seed_worker,
        generator=DATA_LOADER_GEN,
    )

    # Define Metrics
    metrics = {
        "jaccard (IOU)": JaccardBinarySegment(from_logits=True),
        "recall": RecallBinarySegment(from_logits=True),
        "precision": PrecisionBinarySegment(from_logits=True),
        "dice (F1-Score)": DiceBinarySegment(from_logits=True),
    }

    # Define Loss_fn, Optimizer, and Scheduler
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode="min",
        # factor=config["scheduler_factor"],
        factor=hyper_params["scheduler_factor"],
        patience=2,
        min_lr=1e-6,
        threshold=1e-3,
    )

    # train the model (and get the history)
    # TODO: must change the fit_fn to use session.report(...)
    # TODO: use verbose=False to make the output of ray tune clean
    history = fit_fn(
        model,
        train_loader,
        optimizer,
        loss_fn,
        scheduler,
        metrics=metrics,
        val_loader=val_loader,
        epochs=hyper_params["epochs"],
        device=device,
        ray_tune=True,
    )

In [ ]:
# Create a partial func of the model_creator (using functools.partial)
# as we only change the hyperparams of interest for each run of Ray Tune (i.e., hyperparam_tuner)

# Create Tuner Func

In [23]:
def hyperparam_tuner(
    config: dict,
    num_samples: int = 10,
    max_num_epochs: int = 10,
    tune_metric: str = "val_loss",
    tune_metric_mode: str = "min",
):
    """
    Given a model and a set of hyperparameters,
    tries to find the best combination and return the best performing model.

    Parameters
    ----------
    hyper_config: dict
    max_num_epochs: int
    num_samples: int
        Define the total number of samples/combinatin to use to train the model.
        In other words, the number of models to train for finding the best hyperparameters.
    tune_metric: str
        The name of the metric (e.g., val_loss) by which the performance of mdoels are assessed.
    tune_metric_mode: str
        Either "min" (default) or "max", depending on the chosen tune_metric. Specifies whether "min" values are desired, or vice versa.
    """
    # use ASHAScheduler to stop training early on if the model's plight is doomed
    scheduler_ray = ASHAScheduler(
        # metric="loss",  # this is specified in the Tuner's tune.TuneConifg, so only one must be present.
        # mode="min",  # this is specified in the Tuner's tune.TuneConifg, so only one must be present.
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2,
    )

    # method 1 (less control): from Pytorch Tutorials
    # results = tune.run(
    #     model_creator,
    #     resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
    #     config=hyper_config,
    #     num_samples=num_samples,
    #     scheduler=scheduler_ray,
    # )

    # method 2 (more control): from Ray Tune official documentaions
    tuner = tune.Tuner(
        # Use Only CPU
        # model_creator,
        # Use only GPU
        # tune.with_resources(model_creator, {"gpu": 1}),
        # Use both CPU & GPU --> the numbers define how many model to be trained in parallel
        # tune.with_resources(model_creator, {"gpu": 1, "cpu": 0}),  # for my PC
        tune.with_resources(model_creator, {"gpu": 1, "cpu": 12}),  # for Colab Pro A100
        param_space=config,
        tune_config=tune.TuneConfig(
            num_samples=num_samples,
            metric=tune_metric,
            mode=tune_metric_mode,
            search_alg=OptunaSearch(),  # Optuna Works better than Random Search (BOn Pytorch official YouTube)
            scheduler=scheduler_ray,
            chdir_to_trial_dir=False,  # if True (default) the Ray tune will change the current dir, thus no relative path (e.g., path to our data) doesn't work.
        ),
    )
    results = tuner.fit()

    return results

# Run the Tuner

In [29]:
if __name__ == "__main__":
    train_timestamp = datetime.today().strftime(time_format)
    # FIXME: chane num_samples=20 and max_num_epochs=10 for Colab Pro
    results = hyperparam_tuner(
        config=config,
        num_samples=10,  #FIXME: Change this to 60
        max_num_epochs=10,
        # tune_metric="val_dice (F1-Score)",
        # tune_metric_mode="max",
        tune_metric="val_loss",
        tune_metric_mode="min",
    )

/usr/local/lib/python3.10/dist-packages/ray/tune/search/optuna/optuna_search.py:664: FutureWarning: LogUniformDistribution has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :class:`~optuna.distributions.FloatDistribution` instead.
  return ot.distributions.LogUniformDistribution(
[I 2023-07-14 12:20:35,634] A new study created in memory with name: optuna
/usr/local/lib/python3.10/dist-packages/optuna/distributions.py:781: FutureWarning: LogUniformDistribution(high=0.0006, low=0.0005) is deprecated and internally converted to FloatDistribution(high=0.0006, log=True, low=0.0005, step=None). See https://github.com/optuna/optuna/issues/2941.
  warnings.warn(message, FutureWarning)


== Status ==
Current time: 2023-07-14 12:20:35 (running for 00:00:00.11)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource usage: 12.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Result logdir: /root/ray_results/model_creator_2023-07-14_12-20-35
Number of trials: 1/10 (1 PENDING)
+------------------------+----------+-------+--------------+-------------+
| Trial name             | status   | loc   |   image_size |          lr |
|------------------------+----------+-------+--------------+-------------|
| model_creator_857ec677 | PENDING  |       |          352 | 0.000591353 |
+------------------------+----------+-------+--------------+-------------+


== Status ==
Current time: 2023-07-14 12:20:40 (running for 00:00:05.13)
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 16.000: None | Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Logical resource

(pid=132854) 2023-07-14 12:20:42.402913: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Epoch 1:   0%|          | 0/119 [00:00<?, ?it/s]


(model_creator pid=132854) ---------------------------------- epoch 1/10 ---------------------------------


Epoch 1:   8%|▊         | 9/119 [00:01<00:17,  6.24it/s]


...

== Status ==
Current time: 2023-07-14 12:33:51 (running for 00:13:16.10)
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 16.000: None | Iter 8.000: -0.05415083366361531 | Iter 4.000: -0.057689528234980324 | Iter 2.000: -0.06633812104436485 | Iter 1.000: -0.0895263481546532
Logical resource usage: 12.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Current best trial: 857ec677 with val_loss=0.05246283706616272 and parameters={'lr': 0.0005913525136932737, 'image_size': 352}
Result logdir: /root/ray_results/model_creator_2023-07-14_12-20-35
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+------------------------+------------+--------------------+--------------+-------------+--------+------------------+---------------------+--------------+-----------------+
| Trial name             | status     | loc                |   image_size |          lr |   iter |   total time (s) |   val_jaccard (IOU) |   val_recall |   val_precision |
|------------------------+------------+--------------

Epoch 1: 100%|██████████| 119/119 [00:17<00:00,  6.76it/s]


== Status ==
Current time: 2023-07-14 12:33:56 (running for 00:13:21.13)
Using AsyncHyperBand: num_stopped=8
Bracket: Iter 16.000: None | Iter 8.000: -0.05415083366361531 | Iter 4.000: -0.057689528234980324 | Iter 2.000: -0.06633812104436485 | Iter 1.000: -0.0895263481546532
Logical resource usage: 12.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Current best trial: 857ec677 with val_loss=0.05246283706616272 and parameters={'lr': 0.0005913525136932737, 'image_size': 352}
Result logdir: /root/ray_results/model_creator_2023-07-14_12-20-35
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+------------------------+------------+--------------------+--------------+-------------+--------+------------------+---------------------+--------------+-----------------+
| Trial name             | status     | loc                |   image_size |          lr |   iter |   total time (s) |   val_jaccard (IOU) |   val_recall |   val_precision |
|------------------------+------------+--------------

2023-07-14 12:34:14,153	INFO tune.py:1111 -- Total run time: 818.54 seconds (818.50 seconds for the tuning loop).


Result for model_creator_6935b42f:
  date: 2023-07-14_12-34-14
  done: true
  hostname: 1d5d55243a2a
  iterations_since_restore: 1
  node_ip: 172.28.0.12
  pid: 132854
  time_since_restore: 40.093228816986084
  time_this_iter_s: 40.093228816986084
  time_total_s: 40.093228816986084
  timestamp: 1689338054
  training_iteration: 1
  trial_id: 6935b42f
  val_dice (F1-Score): 0.7789245870980349
  val_jaccard (IOU): 0.6390755772590637
  val_loss: 0.09208160571076653
  val_precision: 0.7829081605781208
  val_recall: 0.7764906124635176
  
Trial model_creator_6935b42f completed.
== Status ==
Current time: 2023-07-14 12:34:14 (running for 00:13:38.51)
Using AsyncHyperBand: num_stopped=9
Bracket: Iter 16.000: None | Iter 8.000: -0.05415083366361531 | Iter 4.000: -0.057689528234980324 | Iter 2.000: -0.06633812104436485 | Iter 1.000: -0.09036323241889477
Logical resource usage: 12.0/12 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:A100)
Current best trial: 857ec677 with val_loss=0.05246283706616272 a

# Report the best found hyperparams
> You can find the important reports of ... in:
* ~/ray_results

> Just make sure to delete them if don't need them anymore

> The files include:
1. The result.json (of the best model)
1. The saved checkpoint (of the best model)
1. The parameters (of the best model)

In [32]:
# Get the configs for the best performing model
best_trial = results.get_best_result("val_loss", 'min', "last")
print(f"Best trial config: {best_trial.config}")

# Define the used metrics
metrics = ["loss", "jaccard (IOU)", "dice (F1-Score)", "recall", "precision"]
# add "val_" at the beginning
metrics_val = [f'val_{metric}' for metric in metrics]

for metric in metrics_val:
    print(f"Best trial final {metric}: {best_trial.metrics[metric]}")

Best trial config: {'lr': 0.0005913525136932737, 'image_size': 352}
Best trial final val_loss: 0.05246283706616272
Best trial final val_jaccard (IOU): 0.6851789409464056
Best trial final val_dice (F1-Score): 0.8127096132798628
Best trial final val_recall: 0.7929741279645399
Best trial final val_precision: 0.8353277146816254


In [33]:
# print the results as a DataFrame
df_results = results.get_dataframe()
df_results.insert(loc=0, column="Model", value=range(1, len(df_results.index) + 1))
df_results.to_csv(f"outputs{os.sep}hyperparams{os.sep}{train_timestamp}@ray_tuner_hyperparams.csv", index=False)
print(" Metrics During Training ".center(79, " "))
display(df_results)
# del df_results

                            Metrics During Training                            


,Model,val_jaccard (IOU),val_recall,val_precision,val_dice (F1-Score),val_loss,time_this_iter_s,done,training_iteration,trial_id,...,timestamp,time_total_s,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/image_size,config/lr,logdir
0,1,0.685179,0.792974,0.835328,0.812710,0.052463,40.551349,False,10,857ec677,...,1689337646,402.979904,132854,1d5d55243a2a,172.28.0.12,402.979904,10,352,0.000591,/root/ray_results/model_creator_2023-07-14_12-...
1,2,0.645430,0.793305,0.776334,0.783922,0.079499,41.352243,True,1,e0b649df,...,1689337687,41.352243,132854,1d5d55243a2a,172.28.0.12,41.352243,1,352,0.000584,/root/ray_results/model_creator_2023-07-14_12-...
2,3,0.630841,0.724049,0.831787,0.772585,0.091200,40.708073,True,1,54abb7f6,...,1689337728,40.708073,132854,1d5d55243a2a,172.28.0.12,40.708073,1,352,0.000556,/root/ray_results/model_creator_2023-07-14_12-...
3,4,0.644257,0.761279,0.807615,0.782771,0.088430,41.088845,True,1,9254af0a,...,1689337769,41.088845,132854,1d5d55243a2a,172.28.0.12,41.088845,1,352,0.000508,/root/ray_results/model_creator_2023-07-14_12-...
4,5,0.638430,0.742740,0.820191,0.778557,0.089526,40.319600,True,1,e6c58488,...,1689337810,40.319600,132854,1d5d55243a2a,172.28.0.12,40.319600,1,352,0.000598,/root/ray_results/model_creator_2023-07-14_12-...
5,6,0.650883,0.827459,0.753763,0.788005,0.067129,40.619784,True,2,90ab8fa4,...,1689337891,81.245169,132854,1d5d55243a2a,172.28.0.12,81.245169,2,352,0.000528,/root/ray_results/model_creator_2023-07-14_12-...
6,7,0.598729,0.667056,0.854649,0.747308,0.116395,41.055154,True,1,fb40023b,...,1689337932,41.055154,132854,1d5d55243a2a,172.28.0.12,41.055154,1,352,0.000518,/root/ray_results/model_creator_2023-07-14_12-...
7,8,0.640808,0.821707,0.745465,0.780600,0.101138,40.688427,True,1,0ad750b0,...,1689337973,40.688427,132854,1d5d55243a2a,172.28.0.12,40.688427,1,352,0.000567,/root/ray_results/model_creator_2023-07-14_12-...
8,9,0.636311,0.757653,0.799855,0.776797,0.093513,40.819770,True,1,8727d96a,...,1689338014,40.819770,132854,1d5d55243a2a,172.28.0.12,40.819770,1,352,0.000504,/root/ray_results/model_creator_2023-07-14_12-...
9,10,0.639076,0.776491,0.782908,0.778925,0.092082,40.093229,True,1,6935b42f,...,1689338054,40.093229,132854,1d5d55243a2a,172.28.0.12,40.093229,1,352,0.000562,/root/ray_results/model_creator_2023-07-14_12-...


# Plot the Metrics (for all of the trained models)

In [ ]:
# Obtain a trial dataframe from all run trials of this `tune.run` call.
# dfs = {result: results.get_dataframe()[result] for result in results.get_dataframe()}

# plt.plot(dfs['val_loss'])

# Plot by epoch
# ax = None  # This plots everything on the same plot
# for d in dfs.values():
    # ax = d['val_loss'].plot(ax=ax, legend=False)
    # plt.plot(d['val_loss'])
# ax.set_xlabel('Epochs')
# ax.set_ylabel("val_loss")

# Save the results
> Results are the config and metrics of all the trained models (not just the best one)

> We can use results later on to report different performance in table.

In [ ]:
# save hyperparams as a JSON file
#TODO: Check the type of results
hyper_params_export_name = (
    f"outputs{os.sep}hyperparams_search{os.sep}{train_timestamp}@ray_tuner_hyperparams.json"
)
with open(hyper_params_export_name, "w") as f:
    json.dump(dict(map(lambda x: (x[0], str(x[1])), results)), f)

# save the hyperparams to a csv file
# df_hyperparam = pd.DataFrame(hyper_params, index=[0]).T
# df_hyperparam.to_csv(f'outputs/hyperparams/{start_train_time}@hyperparams.csv', index=True, header=None)
# print('The HyperParameters'.ljust(79, " "))
# display(df_hyperparam)
# del df_hyperparam